# Importing Data from Fitbit 
Fitbit offers an export option for all of a user's data. 
This data is exported in multiple folders, each containing multiple JSON and .csv files. 
For this project, I am interested in the following data: 
- bpm 
- sleep type (heavy, light, REM) and length of each sleep type
- sleep score 
- step data
- active minutes (light, moderate, and very active minutes) 
- resting heartrate
- outside temperature at bedtime (10pm local) 

Each section has a 2 row sample of the dataframe at the end. 

The data is all time based. The goal is to reduce all of the data to get a summary for each day. 
For example, the bpm data is recorded every 5 seconds, and I will reduce it to a daily summary.  

**All resulting dataframes will have one datapoint per day, and will be indexed by date (US/Pacific).**

All of the data is then pickled (as loading from the pickle will be much faster than re-reading the JSON). 
Resulting files used in the analysis will be: 
- bpm.pkl 
- sleep_score.pkl
- sedentary_minutes.pkl
- lightly_active.pkl
- moderately_active.pkl
- very_active.pkl
- step_daily.pkl
- resting_heartrate.pkl

** When you're all done, make sure each data frame has the expect # rows and the expected datatype
Make sure each pickled file exists, and also that its being sourced NOT from the test dir 

Tables:
-bpm
-sleep levels 
-sleep score (done)
-steps (done)
-sedentary minutes (done)
-lightly active minutes (done)
-moderately active minutes (done)
-very active minutes (done)
-resting heartrate (done)
-temperature 
-etc.?
merge all the tables together using date as the primary key 
daily sleep will be the response variable 
everything else will be explanatory variables 



In [236]:
import os
import pandas as pd
import numpy as np 
import datetime as dt 

In [ ]:
# Directory where most of this data lives 
phys_dir = '/Users/jackiekinsler/projects/sleep_analysis_py/physical_data/Physical_Activity'

### IMPORT FROM JSON FUNCTION
This function will help import and concatenate the many JSON files that compromise each data type 

In [297]:
def import_data_from_dir(file_prefix, directory):
    """Reads JSON file(s) in a folder and returns a single dataframe. 
    Takes strings of file_prefix and directory as input. 
    """
    dfs = []
    for file in os.listdir(directory):
        if file_prefix in file: 
            dfs.append(pd.read_json(f"{directory}/{file}"))
    return pd.concat(dfs)

### BPM
Import the bpm data from JSON. 
The JSON data contains a date field and a'value' field. 
The 'value' field contains a dictionary with 'bpm' and 'confidence'. 
The data is imported, the nested 'value' column is unnested. 
The index is also reset, as the index values are not unique.

The data is taken every 5 seconds. The data will be reduced to get daily values for max_bpm and average_bpm. 

In [ ]:
bpm_nested = import_data_from_dir('heart_rate-', '/Users/jackiekinsler/projects/sleep_analysis_py/physical_data/heart_rate')
bpm_nested.to_pickle("bpm_nested.pkl")

In [121]:
# Index needs to be reset as there are repeated values
bpm_nested.reset_index(inplace = True)

In [153]:
bpm_explode = pd.json_normalize(bpm_nested['value'])

In [157]:
# Here, two columns are brought together: the dateTime column from bpm_nested, 
# and the two exploded columns (bpm, confidence) that makeup bpm_explode 
bpm = pd.concat([bpm_nested['dateTime'], bpm_explode], axis = 1)

In [221]:
bpm.head(2)

,dateTime,bpm,confidence
0,2022-03-10 08:00:08,54,3
1,2022-03-10 08:00:13,53,2


In [159]:
bpm.to_pickle("bpm.pkl")

### SLEEP DETAIL
Import the detailed sleep data from JSON.  
The raw JSON has many columns, but the 'levels' columnn is perhaps the most interesting.  
The 'levels' column contains a dictionary of data about the amount of time spent in each sleep type.  
Sleep types include:
- deep
- wake
- light
- REM 

The 'levels' column will be unnested and added back to the dataframe.  
It is important to note that there may be multiple sleep entries for a given day (for example: if there was a long waking period in the middle of sleeping). 

The sleep details for these days will be aggregated into one day. 

In [1010]:
# Import from the sleep directory 
sleep_nested = import_data_from_dir('sleep-', '/Users/jackiekinsler/projects/sleep_analysis_py/physical_data/Sleep')

# There are some entries that are recorded twice... this is because they are in two of the JSON datasets 
# Remove duplicate logId entries 
sleep_nested.drop_duplicates(subset=['logId'], inplace=True)
# Index has duplicate values, so reset_index 
sleep_nested.reset_index(inplace=True, drop=True)

In [1035]:
sleep_nested.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1673 entries, 0 to 1672
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   logId                1673 non-null   int64 
 1   dateOfSleep          1673 non-null   object
 2   startTime            1673 non-null   object
 3   endTime              1673 non-null   object
 4   duration             1673 non-null   int64 
 5   minutesToFallAsleep  1673 non-null   int64 
 6   minutesAsleep        1673 non-null   int64 
 7   minutesAwake         1673 non-null   int64 
 8   minutesAfterWakeup   1673 non-null   int64 
 9   timeInBed            1673 non-null   int64 
 10  efficiency           1673 non-null   int64 
 11  type                 1673 non-null   object
 12  infoCode             1673 non-null   int64 
 13  logType              1673 non-null   object
 14  levels               1673 non-null   object
 15  mainSleep            1673 non-null   bool  
dtypes: boo

In [1011]:
sleep_nested[sleep_nested['dateOfSleep'] =='2019-07-08']


,logId,dateOfSleep,startTime,endTime,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,efficiency,type,infoCode,logType,levels,mainSleep
557,23005269926,2019-07-08,2019-07-07T19:46:30.000,2019-07-08T03:00:00.000,25980000,0,390,43,0,433,94,stages,0,auto_detected,"{'summary': {'deep': {'count': 4, 'minutes': 1...",True


In [1025]:
sleep_nested[sleep_nested['logId'] == 15426414169]

,logId,dateOfSleep,startTime,endTime,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,efficiency,type,infoCode,logType,levels,mainSleep
1194,15426414169,2017-08-05,2017-08-04T23:29:30.000,2017-08-05T09:31:30.000,36120000,0,540,62,1,602,95,stages,0,auto_detected,"{'summary': {'deep': {'count': 10, 'minutes': ...",True


In [1012]:
# Extract the nested data from the 'levels' column 
levels = pd.json_normalize(df_sleep_nested['levels'])

In [1034]:
levels.head(2)

,data,shortData,summary.deep.count,summary.deep.minutes,summary.deep.thirtyDayAvgMinutes,summary.wake.count,summary.wake.minutes,summary.wake.thirtyDayAvgMinutes,summary.light.count,summary.light.minutes,summary.light.thirtyDayAvgMinutes,summary.rem.count,summary.rem.minutes,summary.rem.thirtyDayAvgMinutes,summary.restless.count,summary.restless.minutes,summary.awake.count,summary.awake.minutes,summary.asleep.count,summary.asleep.minutes
0,"[{'dateTime': '2021-05-27T23:06:30.000', 'leve...","[{'dateTime': '2021-05-27T23:24:00.000', 'leve...",7.0,77.0,90.0,31.0,74.0,58.0,34.0,288.0,227.0,7.0,57.0,91.0,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'dateTime': '2021-05-26T23:59:30.000', 'leve...","[{'dateTime': '2021-05-27T01:52:00.000', 'leve...",5.0,119.0,89.0,30.0,69.0,58.0,29.0,215.0,227.0,8.0,128.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN


In [1014]:
# Here, desired columns from the sleep_nested dataframe and the levels dataframe are combined  
sleep_detail_dups = pd.concat([sleep_nested.loc[:,['dateOfSleep','minutesAsleep', 'minutesAwake', 'mainSleep']], levels.loc[:,'summary.deep.count':'summary.rem.thirtyDayAvgMinutes']], axis = 1)
# Set the datatype of the 'dateOfSleep' column to datetime 


In [1017]:
sleep_detail_dups.head(27)

,dateOfSleep,minutesAsleep,minutesAwake,mainSleep,summary.deep.count,summary.deep.minutes,summary.deep.thirtyDayAvgMinutes,summary.wake.count,summary.wake.minutes,summary.wake.thirtyDayAvgMinutes,summary.light.count,summary.light.minutes,summary.light.thirtyDayAvgMinutes,summary.rem.count,summary.rem.minutes,summary.rem.thirtyDayAvgMinutes
0,2019-05-09,408,75,True,7.0,77.0,90.0,31.0,74.0,58.0,34.0,288.0,227.0,7.0,57.0,91.0
1,2019-05-08,423,37,True,5.0,119.0,89.0,30.0,69.0,58.0,29.0,215.0,227.0,8.0,128.0,89.0
2,2019-05-07,455,63,True,5.0,69.0,90.0,23.0,49.0,58.0,26.0,200.0,228.0,8.0,91.0,89.0
3,2019-05-05,88,13,False,4.0,67.0,91.0,33.0,59.0,58.0,35.0,248.0,227.0,7.0,62.0,90.0
4,2019-05-05,434,78,True,3.0,91.0,91.0,37.0,65.0,58.0,36.0,273.0,225.0,9.0,88.0,90.0
5,2019-05-04,392,66,True,6.0,92.0,91.0,36.0,47.0,58.0,32.0,207.0,226.0,12.0,125.0,89.0
6,2019-05-03,421,59,True,5.0,95.0,90.0,34.0,51.0,58.0,36.0,219.0,227.0,8.0,85.0,89.0
7,2019-05-02,405,43,True,3.0,64.0,92.0,26.0,30.0,60.0,26.0,209.0,227.0,6.0,61.0,90.0
8,2019-05-01,365,65,True,4.0,101.0,91.0,32.0,62.0,60.0,32.0,240.0,227.0,7.0,76.0,91.0
9,2019-04-30,431,63,True,3.0,95.0,91.0,31.0,53.0,60.0,31.0,217.0,227.0,7.0,109.0,90.0


In [1016]:
sleep_detail_dups['dateOfSleep'] = pd.to_datetime(sleep_detail_dups['dateOfSleep'])

In [1006]:
# For nights with more than one sleep, sum the sleep statistics from that night 
sleep_detail = sleep_detail_dups.groupby(by='dateOfSleep').sum()
# Because of the summation, NaN values will be summed to zero. 
# Many nights have 0.0 as the value, because Fitbit was unable to get detailed enough 
# Sleep data to calculate different sleep levels. 

In [1009]:
sleep_detail.head(27)

,minutesAsleep,minutesAwake,mainSleep,summary.deep.count,summary.deep.minutes,summary.deep.thirtyDayAvgMinutes,summary.wake.count,summary.wake.minutes,summary.wake.thirtyDayAvgMinutes,summary.light.count,summary.light.minutes,summary.light.thirtyDayAvgMinutes,summary.rem.count,summary.rem.minutes,summary.rem.thirtyDayAvgMinutes
dateOfSleep,,,,,,,,,,,,,,,
2017-07-25,471,80,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-30,237,23,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-02,444,84,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-05,540,62,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-18,163,18,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-09-01,105,7,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-09-05,494,34,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-09-06,481,30,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-09-07,498,78,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [967]:
sleep_detail.to_pickle("data/sleep_detail.pkl")

In [1024]:
sleep_detail_dups[sleep_detail_dups['dateOfSleep'] =='2017-08-05']

,dateOfSleep,minutesAsleep,minutesAwake,mainSleep,summary.deep.count,summary.deep.minutes,summary.deep.thirtyDayAvgMinutes,summary.wake.count,summary.wake.minutes,summary.wake.thirtyDayAvgMinutes,summary.light.count,summary.light.minutes,summary.light.thirtyDayAvgMinutes,summary.rem.count,summary.rem.minutes,summary.rem.thirtyDayAvgMinutes
1194,2017-08-05,540,62,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1022]:
sleep_detail.loc[['2017-10-02']]

,minutesAsleep,minutesAwake,mainSleep,summary.deep.count,summary.deep.minutes,summary.deep.thirtyDayAvgMinutes,summary.wake.count,summary.wake.minutes,summary.wake.thirtyDayAvgMinutes,summary.light.count,summary.light.minutes,summary.light.thirtyDayAvgMinutes,summary.rem.count,summary.rem.minutes,summary.rem.thirtyDayAvgMinutes
dateOfSleep,,,,,,,,,,,,,,,
2017-10-02,466,102,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### SLEEP SCORE
Sleep score is in a .csv with 1 row per sleep (there may be multiple sleeps per night). 

`overall_sleep` score is a sum of individual scores in sleep duration, sleep quality, and restoration
- Excellent: 90-100
- Good: 80-89
- Fair: 60-79
- Poor: Less than 60
To understand more about the sleep score: https://help.fitbit.com/articles/en_US/Help_article/2439.htm

The resulting dataframe is indexed by date, with `overall_score`, `sleep_resting_heartrate`, and `deep_sleep_in_min` columns. 

In [844]:
sleep_score_full = pd.read_csv('/Users/jackiekinsler/projects/sleep_analysis_py/physical_data/Sleep/sleep_score.csv')
# Keep only the rows of interest
sleep_score_reduced = sleep_score_full.loc[:,['timestamp', 'overall_score', 'deep_sleep_in_minutes', 'resting_heart_rate']]

In [948]:
ss = sleep_detail[sleep_detail['dateOfSleep'] == '2022-02-14'] 
ss

,dateOfSleep,minutesAsleep,minutesAwake,mainSleep,summary.deep.count,summary.deep.minutes,summary.deep.thirtyDayAvgMinutes,summary.wake.count,summary.wake.minutes,summary.wake.thirtyDayAvgMinutes,summary.light.count,summary.light.minutes,summary.light.thirtyDayAvgMinutes,summary.rem.count,summary.rem.minutes,summary.rem.thirtyDayAvgMinutes
36,2022-02-14,125,5,False,5.0,111.0,86.0,26.0,58.0,61.0,27.0,173.0,256.0,6.0,97.0,76.0
37,2022-02-14,270,46,True,4.0,101.0,85.0,33.0,60.0,61.0,31.0,243.0,257.0,6.0,85.0,75.0


In [845]:
# Convert timestamp to a date, and then remove the time portion of the timestamp
sleep_score_reduced['timestamp'] = pd.to_datetime(sleep_score_reduced['timestamp']).dt.normalize()

It is possible to have more than one entry per date (there may be multiple sleeps per night).  

Below, the data is grouped by date. Then, a weighted average of the `overall_score` and `resting_heart_rate` is taken using the `deep_sleep_in_minutes` column. 

The `deep_sleep_in_minutes` is simply summed.
This results in a table with unique dates. 
The date is then used as the index. 

In [895]:
# Group entries by date 
grouped_by_time = sleep_score_reduced.groupby(by='timestamp')

# Here, we return a series to maintain the column name (would be lost otherwise)
overall_score = grouped_by_time.apply(
        lambda x: 
        pd.Series({
            'overall_score' : np.average(x.overall_score, weights=x.deep_sleep_in_minutes)
        })
    )
sleep_resting_heartrate = grouped_by_time.apply(
        lambda x: 
        pd.Series({ 
            'sleep_resting_heartrate' : np.average(x.resting_heart_rate, weights=x.deep_sleep_in_minutes)
        })
    )
deep_sleep_in_min = pd.DataFrame(grouped_by_time['deep_sleep_in_minutes'].sum())

In [896]:
# Concatenate the three columns into a new dataframe 
sleep_score = pd.concat([overall_score, sleep_resting_heartrate, deep_sleep_in_min], axis=1)

In [898]:
sleep_score.head(2)

,overall_score,sleep_resting_heartrate,deep_sleep_in_minutes
timestamp,,,
2019-08-22 00:00:00+00:00,83.0,48.0,102
2019-08-29 00:00:00+00:00,73.0,52.0,93


In [901]:
sleep_score.to_pickle("data/sleep_score.pkl")

### STEPS
Import steps data from JSON.  
Steps data is recorded every few minutes. The number of steps for that period of time is recorded.   
The data will be reduced to the total number of steps for each day.   

Resulting table `step_daily` is date (in US/Pacific), and the total number of steps on that date.

In [359]:
step_detail = import_data_from_dir('steps-', phys_dir)
step_detail.to_pickle("data/step_detail.pkl")

In [562]:
step_detail = pd.read_pickle('data/step_detail.pkl')
# Convert the dateTime column from UTC to Pacific 
step_detail['dateTime'] = step_detail['dateTime'].dt.tz_localize('UTC').dt.tz_convert('US/Pacific')

In [563]:
# Get the sum of steps for each day 
step_daily = step_detail.groupby([step_detail['dateTime'].dt.date]).sum()
# .groupby turns the grouped column (dateTime) into the index.
# Use .to_datetime() to make it a DatetimeIndex
step_daily.index = pd.to_datetime(step_daily.index)
step_daily.to_pickle('data/step_daily.pkl')

In [564]:
step_daily.head(2)

,value
dateTime,
2017-07-19,4040
2017-07-20,9033


In [565]:
# Sample of getting a step value for a specific date 
step_daily.loc['2022-12-9']

value    11571
Name: 2022-12-09 00:00:00, dtype: int64

### ACTIVITY MINUTES

Activity minutes are imported from JSON. 
The documentation does not provide information about what time zone the dateTime stamp is from. 
By looking at the data and aligning it with known activity on different days, I am making the assumption that the data is recorded in local time. Although, if it was recorded in UTC, the data would be off by 1 day.

In [715]:
def import_activity_min(file_prefix, directory):
    """
    For activity minutes only! Takes a file_prefix and directory, and returns a data table 
    with a DatetimeIndex and a value for activity minutes for that day. 
    """
    df = import_data_from_dir(file_prefix, directory)
    df.dropna(how='any', inplace=True)
    # Create a DatetimeIndex from the dateTime column, then drop the original dateTime column 
    df.set_index(pd.DatetimeIndex(df['dateTime']), inplace=True)
    df.drop('dateTime', axis=1, inplace=True)
    return df 

In [717]:
# Importing activity minutes 
sedentary_minutes = import_activity_min('sedentary_minutes', phys_dir)
lightly_active = import_activity_min('lightly_active', phys_dir)
moderately_active = import_activity_min('moderately_active', phys_dir)
very_active = import_activity_min('very_active_minutes', phys_dir)

# Pickle the data for future use 
sedentary_minutes.to_pickle("data/sedentary_minutes.pkl")
lightly_active.to_pickle("data/lightly_active.pkl")
moderately_active.to_pickle("data/moderately_active.pkl")
very_active.to_pickle("data/very_active.pkl")

In [726]:
moderately_active.head(2)

,value
dateTime,
2017-07-18,0
2017-07-19,37


In [725]:
# Sample of getting a value for a specific date 
moderately_active.loc['07/29/17']

value    11
Name: 2017-07-29 00:00:00, dtype: int64

### RESTING HEARTRATE

Import resting heartrate from JSON. 

The final table `resting heartrate` will be indexed by date (in US/Pacific), with a heartrate value, and error. 

In [684]:
# Importing resting_heartrate data 
resting_heartrate_nested = import_data_from_dir('resting_heart_rate', phys_dir)
# The data in this dataframe is nested, and only the last column ('value') has the needed data 
resting_heartrate = pd.json_normalize(resting_heartrate_nested['value'])
# Drop any rows with a NaN value 
resting_heartrate.dropna(how='any', inplace=True)
# Make 'date' the index, and convert it to a Datetime data type 
resting_heartrate.set_index(pd.DatetimeIndex(resting_heartrate['date']), inplace=True)
# Drop the old date column 
resting_heartrate.drop('date', axis=1, inplace=True)

resting_heartrate.to_pickle("data/resting_heartrate.pkl")

In [678]:
resting_heartrate.head(2)

,value,error
date,,
2022-07-29,52.782105,26.761181
2022-07-30,50.288383,10.639998


In [683]:
# Sample of getting a resting_heartrate value for a specific date 
resting_heartrate.loc['2022-11-11']

value    53.142025
error    10.854312
Name: 2022-11-11 00:00:00, dtype: float64